In [ ]:
%%capture
%pip install datasets transformers[tf,torch,sentencepiece,vision,optuna,sklearn,onnxruntime]==4.11.3


In [ ]:
#hide
from utils import *
setup_chapter()

In this book we will demonstrate how you can run the example from the book in Amazon SageMaker. 

The SageMaker notebook uses an AWS IAM role to access AWS resources such as Amazon S3 bucket.
You created this role during the notebook creation process described in the README.md in SageMaker/README.md.
In the AWS IAM service you are able to review the access policy and you can modify it.

In the next cell we will check an Amazon S3 bucket exists and create a new one if not. In addition we'll get the SageMaker role and session.

In [3]:
!pip install sagemaker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.0/748.0 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import sagemaker.huggingface
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker role arn: arn:aws:iam::123456789012:role/my-sagemaker-role
sagemaker bucket: my-sagemaker-bucket
sagemaker session region: us-west-2


Now we setup a helper function to easily deploy any Hugging Face model as an endpoint on AWS SageMaker.
We use the following function to create a HuggingFaceModel Class, where we are going to download the model from the Hugging Face hub. This class would also allow to use a trained model stored on the Amazon S3 bucket. 
Next, an endpoint will be created and this endpoint will host your model. Based on the model requirements you can choose a specific instance type which are equipped differently in memory, cpu, gpu. There are different inferences available, such as real-time, asynchronous or serverless.
If you are not sure which inference works best for your model, you use Amazon SageMaker Inference Recommender.
https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html 

To view all options see the documentation: https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/index.html 

Depending on Transformer version, PyTorch/TensorFlow version and Python version, the mapping for the Hugging Face Model Class can be found here: https://huggingface.co/docs/sagemaker/reference#inference-dlc-overview 

To find the endpoints in the AWS Console navigate to https://console.aws.amazon.com/sagemaker/home#/endpoints 

Make sure to finish this notebook to delete the endpoint in the end. 

In [4]:
from sagemaker.huggingface.model import HuggingFaceModel

def setup_endpoint(model_name, task_name):
    # Hub Model configuration. <https://huggingface.co/models>
    hub = {
      'HF_MODEL_ID': model_name, # model_id from hf.co/models
      'HF_TASK': task_name # NLP task you want to use for predictions
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
       env=hub, # configuration for loading model from Hub
       role=role, # iam role with permissions to create an Endpoint
       transformers_version="4.17.0", # transformers version used
       pytorch_version="1.10.2", # pytorch version used
       py_version="py38" # python version used
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
       initial_instance_count=1, # how many instances used
       instance_type="ml.m5.xlarge" # instance type
    )
    return predictor

# Hello Transformers

## The Encoder-Decoder Framework

## Attention Mechanisms

## Transfer Learning in NLP

## Hugging Face Transformers: Bridging the Gap

## A Tour of Transformer Applications

In [5]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

### Text Classification

In [6]:
predictor = setup_endpoint('distilbert-base-uncased-finetuned-sst-2-english', 'text-classification')

In [6]:
# example request, you always need to define "inputs"
import pandas as pd

# request
outputs = predictor.predict({"inputs": text})
df=pd.DataFrame(outputs)    

In [6]:
predictor.delete_endpoint()

### Named Entity Recognition

In [6]:
predictor = setup_endpoint("dbmdz/bert-large-cased-finetuned-conll03-english", "ner")

In [6]:
outputs = predictor.predict({"inputs": text, "parameters": {"aggregation_strategy": "simple"}})
df=pd.DataFrame(outputs)    

In [6]:
predictor.delete_endpoint()

### Question Answering 

In [6]:
predictor = setup_endpoint("distilbert-base-cased-distilled-squad", 'question-answering')

In [6]:
question = "What does the customer want?"

outputs = predictor.predict({"inputs": {
    "question": question,
    "context": text
    }
})

df=pd.DataFrame([outputs])    

In [6]:
predictor.delete_endpoint()

### Summarization

In [6]:
predictor = setup_endpoint("sshleifer/distilbart-cnn-12-6", 'summarization')

In [7]:
outputs = predictor.predict({"inputs": text,
                             "parameters": {
                                 "max_length":45,
                                 "clean_up_tokenization_spaces":True
                                 }
                            })
print(outputs[0]['summary_text'])

Dear Amazon, I ordered an Optimus Prime AF online from Germany, but received a Megatron AF instead. As a lifelong enemy of the Decepticons, I need an exchange of Megatron for the correct figure. I have included copies of my purchase records and look forward to hearing from you soon.Sincerely, Bumblebee


In [ ]:
predictor.delete_endpoint()

### Translation

In [ ]:
predictor = setup_endpoint("Helsinki-NLP/opus-mt-en-de", "translation")

In [8]:
outputs = predictor.predict({"inputs": text,
                             "parameters": {
                                 "min_length":100,
                                 "clean_up_tokenization_spaces":True
                                 }
                            })
print(outputs[0]['translation_text'])

Sehr geehrtes Amazon, ich habe eine Optimus Prime AF online aus Deutschland bestellt, habe jedoch stattdessen eine Megatron AF erhalten. Als lebenslanger Feind der Decepticons benötige ich einen Umtausch von Megatron gegen die richtige Figur. Ich habe Kopien meiner Kaufaufzeichnungen beigefügt und freue mich auf Ihre baldige Antwort.Mit freundlichen Grüßen, Bumblebee.


In [ ]:
predictor.delete_endpoint()

### Text Generation

In [ ]:
predictor = setup_endpoint("gpt2", 'text-generation')

In [9]:
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response

outputs = predictor.predict({"inputs": prompt,
                             "parameters": {
                                 "max_length":200
                                 }
                            })
print(outputs[0]['generated_text'])

Thank you for reaching out to us, Bumblebee. We're sorry to hear about the mix-up with your order, and we understand how frustrating this can be. Please provide us with your order number and any other relevant information so we can investigate the issue and find a solution for you.


In [ ]:
predictor.delete_endpoint()

## The Hugging Face Ecosystem

### The Hugging Face Hub

### Hugging Face Tokenizers

### Hugging Face Datasets

### Hugging Face Accelerate

## Main Challenges with Transformers

## Conclusion